In [1]:
# Import Libraries
import json
import minsearch
from openai import OpenAI

In [2]:
# Read JSON file and extract document details

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [5]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [6]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [7]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [11]:
llm('i have problems with docker. Please assist')

" I'd be glad to help you troubleshoot your Docker issues! To get started, could you please provide more details about the specific problem or error message you are encountering? Understanding what goes wrong will allow me to offer targeted solutions and guidance for resolving common pitfalls in using Docker.\n\nOnce I have this information, we can explore potential fixes such as verifying your system requirements (Docker needs a minimum of 4GB RAM), ensuring that you've installed the correct version compatible with your operating system, checking network connectivity if interacting over it, and inspecting any permissions or file/directory access issues. Remember to also ensure Docker is running correctly before proceedsuing by using commands like `docker ps` (to list all containers) and `docker info`.\n\nBy sharing the error message with me along with some context around what you're trying to do, we can troubleshoot together more effectively!"